In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
currdir = os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)
print(currdir)
lines_to_skip = 10 # adjust this as necessary

# count the number of header lines
header_lines = 3 # adjust this as necessary

plate_type = 'DIV12'
subfolder = 'LSL-Cas9-PVcre_5thbatch'
# read the csv file into a pandas DataFrame, skipping the metadata at the top
# df = pd.read_csv(filename, skiprows=lines_to_skip, header=[i for i in range(header_lines)])
### read all data
dose1_folder = f"{parent}/data/DIV19_CNO/Div19_5uM_CNO"

exp1 = pd.read_csv(f'{parent}/data/{plate_type}/{subfolder}/DIV 12 LSL-Cas9-PVcre with fith batch of gRNAs plate1(000)(000)_CompiledData.csv',
                   engine='python', skiprows = lines_to_skip)
exp2 = pd.read_csv(f'{parent}/data/{plate_type}/{subfolder}/DIV 12 LSL-Cas9-PVcre with fith batch of gRNAs plate2(000)(000)_CompiledData.csv',
                   engine='python', skiprows = lines_to_skip)
exp3 = pd.read_csv(f'{parent}/data/{plate_type}/{subfolder}/DIV 12 LSL-Cas9-PVcre with fith batch of gRNAs plate1(000)(000)_CompiledData.csv',
                   engine='python',
                  skiprows = lines_to_skip)

exp1 = exp1.drop(columns=['Unnamed: 9'])
exp2 = exp2.drop(columns=['Unnamed: 9'])
exp3 = exp3.drop(columns=['Unnamed: 9'])

##

firstTableHeading = "Mean Firing Rate (Hz)"

/Users/poojap/Documents/SoderlingLab/MEA_analysis/analysis


In [4]:
print(exp1.tail(10))

    Unnamed: 0  GPR37L 1  GPR37L 2  LGI2 A12   SLITRK5   THSD7 1   THSD7 2  \
260        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
261        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
262        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
263        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
264        NaN  8.000000  8.000000  8.000000  7.000000  7.000000  8.000000   
265        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
266        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
267        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
268        NaN  8.000000  8.000000  8.000000  8.000000  8.000000  8.000000   
269        NaN  7.000000  8.000000  7.000000  8.000000  8.000000  8.000000   

         unt1      unt2  
260  8.000000  8.000000  
261  8.000000  8.000000  
262  8.000000  8.000000  
263  8.000000  8.000000  
264  8.0000

In [5]:

def rename_row_names(experiment_df):
    replace_rows = experiment_df['Unnamed: 0'].isin(['B Replicates', np.nan]) 
    replace_count = replace_rows.sum()
    
    replicates = (f"B Replicate {i+1}" for i in range(replace_count))
    with pd.option_context('mode.chained_assignment', None):
        experiment_df.loc[replace_rows, 'Unnamed: 0'] = list(replicates)
    return experiment_df


In [15]:
def section_data(exp_num, firstHeading):
    maxrows = exp_num.shape[0]
    vals_between_tables = 16
    titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
    # print(titles[-1], print(exp_num.iloc[254]))
    # print(maxrows, titles)
    # print(maxrows, titles)
    titles.insert(0,0)

    # # Empty dictionary to store dataframes
    experiment = {}
    
    # Loop through start indices
    for ind in range(len(titles) -1):

        if ind == 0:
            ## Handle mean firing rate
            key = firstHeading
            table = exp_num.iloc[titles[ind] : titles[ind +1], :]
            table = rename_row_names(table)
        elif ind != 0:
            table = exp_num.iloc[titles[ind] : titles[ind +1], :]

            key = table.iloc[0][0]
            table = exp_num.iloc[titles[ind] +2: titles[ind +1], :]

            table = rename_row_names(table)
        table.set_index('Unnamed: 0', inplace=True)
        experiment[key] = table
    return experiment                   
exp1_dict = section_data(exp1, firstTableHeading)
exp2_dict = section_data(exp2, firstTableHeading)
exp3_dict = section_data(exp3, firstTableHeading)
exp3_dict.keys()

dict_keys(['Mean Firing Rate (Hz)', 'Number of Active Electrodes', 'Weighted Mean Firing Rate (Hz)', 'Number of Bursting Electrodes', 'Burst Frequency - Avg (Hz)', 'Burst Duration - Avg (s)', 'Normalized Duration IQR - Avg', 'IBI Coefficient of Variation - Avg', 'Burst Percentage - Avg', 'Network Burst Frequency (Hz)', 'Network Burst Duration - Avg (sec)', 'Network Burst Percentage', 'Network IBI Coefficient of Variation', 'Network Normalized Duration IQR', 'Area Under Normalized Cross-Correlation', 'Resistance - Avg (kOhms)', 'Number of Covered Electrodes'])

In [16]:
experiments = [exp1_dict, exp2_dict, exp3_dict]

In [17]:
def df_is_in(sub_df, df):
    # Convert rows of each DataFrame to tuples
    sub_tuples = set(sub_df.apply(tuple, axis=1))
    df_tuples = set(df.apply(tuple, axis=1))
    
    # Check if all tuples of sub_df exist in df
    return sub_tuples.issubset(df_tuples)


In [23]:
def merge_experiments_DIV(list_of_dicts):
    data = {}
    for i, dictionary in enumerate(list_of_dicts):
        data[f"exp{i +1}"] = dictionary
    merged_data = data["exp1"].copy()
    
    for i, (exp, dictionary) in enumerate(data.items()):
        if exp == "exp1":
            continue
        for key, value in dictionary.items():
            merged_data[key] = pd.concat([merged_data[key], value])


    return merged_data

data = merge_experiments_DIV(experiments)
len(list(data.keys()))
data['Mean Firing Rate (Hz)'].shape

(42, 8)

In [24]:
print(data.keys())
print(data['Mean Firing Rate (Hz)'].shape)

# data['exp2']['Mean Firing Rate (Hz)'].shape

# data['Mean Firing Rate (Hz)'].to_csv("data_exp1_meanfiringrate_check.csv")


dict_keys(['Mean Firing Rate (Hz)', 'Number of Active Electrodes', 'Weighted Mean Firing Rate (Hz)', 'Number of Bursting Electrodes', 'Burst Frequency - Avg (Hz)', 'Burst Duration - Avg (s)', 'Normalized Duration IQR - Avg', 'IBI Coefficient of Variation - Avg', 'Burst Percentage - Avg', 'Network Burst Frequency (Hz)', 'Network Burst Duration - Avg (sec)', 'Network Burst Percentage', 'Network IBI Coefficient of Variation', 'Network Normalized Duration IQR', 'Area Under Normalized Cross-Correlation', 'Resistance - Avg (kOhms)', 'Number of Covered Electrodes'])
(42, 8)


In [25]:
def remove_mean_SEM(dictionary):
    for key, table in dictionary.items():
        table.drop(['B Mean', 'B SEM'], axis = 0, inplace=True)
        dictionary[key] = table
    return dictionary

for key, dictionary in data.items():
    data = remove_mean_SEM(dictionary)
# print(basal_dict['Number of Covered Electrodes'].columns)
# dose1_dictmessy['Mean Firing Rate (Hz)']

ValueError: cannot reindex on an axis with duplicate labels

In [26]:

def perform_t_test(exp_df):
    exp_results = {}
    for variable_test in exp_df.keys():
        exp_results[variable_test] = {}
        # print(exp_df[variable_test])
        table = exp_df[variable_test].copy()  # Create a deep copy
        table.drop(['B Mean', 'B SEM'], axis = 0, inplace=True)
        test_variables = [col for col in table.columns if "unt" not in col.lower()]
        control_name = "unt2"
        for testitem in test_variables:
            exp_group = list(table[testitem].astype(float))
            control_group = list(table[control_name].astype(float))
            try:
                with warnings.catch_warnings(record=True) as w:
                    warnings.simplefilter("always")  # Cause all warnings to always be triggered

                    t_stat, pval = stats.ttest_ind(exp_group, control_group)

                    if len(w) > 0:
                        # A warning occurred, add a note to the results
                        exp_results[variable_test][testitem] = [t_stat, pval, 'Warning: ' + str(w[-1].message)]
                    else:
                        # No warning, add results normallytqdm
                        ## first value is t-stat like FC, second is Pvalue
                        exp_results[variable_test][testitem] = [t_stat, pval]
            except RuntimeError as re:
                # Catch any runtime errors and handle them
                print(f'Error: {str(re)}')
    return exp_results

all_experiments_results = perform_t_test(data)
all_experiments_results

{'Mean Firing Rate (Hz)': {'GPR37L 1': [-3.512461934976752,
   0.0007820620538489373],
  'GPR37L 2': [2.882072139616893, 0.005242025933865089],
  'LGI2 A12': [-4.471781454337339, 2.9234062784044448e-05],
  'SLITRK5': [-0.9230712109710678, 0.3591409683703768],
  'THSD7 1': [-7.092952178728836, 8.45292200803454e-10],
  'THSD7 2': [-5.622808074352924, 3.578077329196222e-07]},
 'Number of Active Electrodes': {'GPR37L 1': [-1.4348601079588734,
   0.1557799085449525,
   'Warning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.'],
  'GPR37L 2': [nan,
   nan,
   'Warning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.'],
  'LGI2 A12': [-0.9999999999999967,
   0.32075485777547497,
   'Warning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs

In [12]:
# exp1.iloc[62:79]
def analyze_test_results(experiment_res, outfile):
    col_names = ['Test Types', 'Gene', 'T-statistic', 'Regulation', 'P-value', 'Significance', 'Warnings']
    df = pd.DataFrame(columns = col_names)

    directory = os.path.dirname(outfile)
    if not os.path.exists(directory):
        os.makedirs(directory)

    for test in (experiment_res.keys()):
        for prot in experiment_res[test].keys():
            new_row = {}

            new_row['Test Types'] = test
            new_row['Gene'] = prot
            t_stat = experiment_res[test][prot][0]
            new_row['T-statistic'] = t_stat
            if t_stat < 0:
                new_row['Regulation'] = "Down-Regulated"
            elif t_stat > 0:
                new_row['Regulation'] = "Up-Regulated"
            else:
                new_row['Regulation'] = "Same/NA"
            
            pval = experiment_res[test][prot][1]
            new_row['P-value'] = pval
            if pval <= 0.05:
                new_row['Significance'] = "Significant diff between gene and control"
            else:
                new_row['Significance'] = "Not Significant"
            if len(experiment_res[test][prot]) > 2:
                new_row['Warnings'] = experiment_res[test][prot][2]
            else:
                new_row['Warnings'] = "No Warning"

            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_csv(outfile)
    return df


test_results = analyze_test_results(all_experiments_results, f'predictions/{plate_type}/{subfolder}/all_plates.csv')

In [13]:

def save_all_to_excel(dataframes, outfile):
    # Convert single dataframe to a list of one dataframe for consistency
    if isinstance(dataframes, pd.DataFrame):
        dataframes = [dataframes]
        sheet_names = ['experiment 1']
    else:
        sheet_names = ['experiment {}'.format(i+1) for i in range(len(dataframes))]

    directory = os.path.dirname(outfile)
    if not os.path.exists(directory):
        os.makedirs(directory)
    writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

    for df, sheetname in zip(dataframes, sheet_names):
        df.to_excel(writer, sheet_name=sheetname)
    
    writer.save()
    if os.path.exists(outfile):
        return "File Created"
    else:
        return "Problem in experiments to excel file"

save_all_to_excel(test_results, f"predictions/{plate_type}_3plates.xlsx")

AttributeError: 'XlsxWriter' object has no attribute 'save'

In [17]:
def main():
    exp1_df = section_data(exp1, firstTableHeading)
    exp2_df = section_data(exp2, firstTableHeading)
    exp3_df = section_data(exp3, firstTableHeading)
    ## combine 3 tests to get analysis for all replicates 
    experiments = [exp1_dict, exp2_dict, exp3_dict]
    data = merge_experiments_DIV(experiments)
    all_experiments_results = perform_t_test(data)

    # exp1_results = perform_t_test(exp1_df)
    # exp2_results = perform_t_test(exp2_df)
    # exp3_results = perform_t_test(exp3_df)
    test_results = analyze_test_results(all_experiments_results, f'data/{plate_type}/all_plates.csv')
    save_path = f"predictions/{plate_type}_3plates.xlsx"
    save_all_to_excel(test_results, f"predictions/{plate_type}_3plates.xlsx")    


In [14]:
if __name__ == '__main__':
    main()
    ##################################
    ## the t-test may not be accurate because exp group and control group are very similar in 4 tests

/tmp/ipykernel_2157763/2327352467.py:17: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [21]:
# Save the plots as images
t_statistic_path = f"{currdir}/plots/{plate_type}/t_stat_{plate_type}"
regulation_path = f"{currdir}/plots/{plate_type}/regulation_count.png"
p_value_path = f"{currdir}/plots/{plate_type}/p_value_distribution_{plate_type}.png"
save_path = f"predictions/{plate_type}_3plates.xlsx"

currdir = os.getcwd()
parent = os.path.dirname(currdir)

data = pd.read_excel(currdir + "/" +save_path )
# Saving T-statistic vs. Test Types
plt.figure(figsize=(10, 6))
sns.boxplot(y='Test Types', x='T-statistic', data=data, palette='coolwarm')
plt.title('Distribution of T-statistic Values for Different Test Types')
plt.xlabel('T-statistic')
plt.ylabel('Test Types')
plt.legend(title='T statistic', loc = 'lower right')
plt.tight_layout()
plt.savefig(t_statistic_path)
plt.close()

# Saving Regulation Count for each Test Type
plt.figure(figsize=(10, 6))
sns.countplot(y='Test Types', hue='Regulation', data=data, palette='muted')
plt.title('Number of Up-regulated vs. Down-regulated Genes per Test Type')
plt.xlabel('Number of Genes')
plt.ylabel('Test Types')
plt.legend(title='Regulation', loc='lower right')
plt.tight_layout()
plt.savefig(regulation_path)
plt.close()

# Saving P-value Distribution for each Test Type
plt.figure(figsize=(10, 6))
sns.violinplot(y='Test Types', x='P-value', data=data, palette='viridis', inner='quartile')
plt.title('Distribution of P-values for Different Test Types')
plt.xlabel('P-value')
plt.ylabel('Test Types')
plt.xscale('log')
plt.legend(title='P value', loc = 'lower right')
plt.tight_layout()
plt.savefig(p_value_path)
plt.close()

t_statistic_path, regulation_path, p_value_path


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


('/home/poojaparameswaran/Documents/SoderlingLab/MEA_analysis/plots/DIV19/t_stat_DIV19',
 '/home/poojaparameswaran/Documents/SoderlingLab/MEA_analysis/plots/DIV19/regulation_count.png',
 '/home/poojaparameswaran/Documents/SoderlingLab/MEA_analysis/plots/DIV19/p_value_distribution_DIV19.png')